In [1]:
import pandas as pd
import numpy as np


# The dataset has the following files
- 30 means there were 30 samples per reference point
- avg means the avgs rss values were used for each ap for all 30 samples and put as one sample
- All means they used all the APs for all buildings
- AP means they only used APs present in the building


![image](../pictures/dataset-files.png)

# The number of samples per building is as follows:

![image](../pictures/statistics-about-dataset.png)

# We will train using AP_30 for all buildings

- we will use the given preinstalled ap locations as features for our graph.

# First we need to load the xlsx file for the pre-instaleld AP locations.

Then we create a dictionary with the APs and their locations for each building

In [6]:
pre_installed_aps_all = pd.read_excel("../SODIndoorLoc-main/Information of pre-installed APs.xlsx")
xlsx_file = pd.ExcelFile("../SODIndoorLoc-main/Information of pre-installed APs.xlsx")

In [12]:
# get names of buildings

building_names = xlsx_file.sheet_names

# get the dataframes for each building as dictionary of dataframes with name of building as key

building_dfs = {name: xlsx_file.parse(name) for name in building_names}

In [14]:
# define function to read xlsx file and return dataframe dictionary of all buildings

def read_xlsx_file(filename):
    # read the xlsx file
    data = pd.ExcelFile(filename)
    # return the data
    dfs = {name: data.parse(name) for name in data.sheet_names}
    return dfs


In [77]:
# we only want 2.4 GHz APs
# define a dictionary to get information about the APs for each building

def get_ap_info_per_building(dataframe):

    ap_info_24 = {}
    ap_info_5 = {}
    # loop over dataframe and get the information
    for idx, row in dataframe.iterrows():
        # get the ap name from attribute_2.4 column
        ap_name_24 = row["Attribute_2.4"]
        ap_name_5 = row["Attribute_5"]
        # get ap x and y coords 
        ap_x = row["ECoord"]
        ap_y = row["NCoord"]

        # getr floor ID
        ap_floor_id = row["FloorID"]

        # put info in dictionaries
        ap_info_24[ap_name_24] = {"x": ap_x, "y": ap_y, "floor_id": ap_floor_id}
        ap_info_5[ap_name_5] = {"x": ap_x, "y": ap_y, "floor_id": ap_floor_id}
    
    ap_building = {"2.4": ap_info_24, "5": ap_info_5}
    return ap_building


building_ap_info = {
    building_name: get_ap_info_per_building(building_df) for building_name, building_df in building_dfs.items()
}


In [78]:
# save ap coordinates for 2.4 ghz in csv files for each building

def save_ap_coords(buildings_dict):

    for key in buildings_dict.keys():
        # get the ap coords
        ap_coords = buildings_dict[key]["2.4"]

        aps = []

        for ap in ap_coords.keys():
            aps.append([ap_coords[ap]["x"], ap_coords[ap]["y"]]) # , ap_coords[ap]["floor_id"]

        aps = pd.DataFrame(aps, columns=["x", "y"]) # , "FloorID"
        # save the ap coords
        aps.to_csv("../data/raw/{}_ap_coords.csv".format(key), index=False)


save_ap_coords(building_ap_info)

In [79]:
# we will only use 2.4 GHz APs

# load scans csv file for each building and drop the 5 GHz APs

# load SYL 
syl_scans_train = pd.read_csv("../SODIndoorLoc-main/SYL/Training_SYL_AP_30.csv")
syl_scans_test = pd.read_csv("../SODIndoorLoc-main/SYL/Testing_SYL_AP.csv")

In [80]:
# get 5 GHz AP names

syl_5ghz_aps_names = [key for key in building_ap_info["SYL"]["5"].keys()]

# drop the 5 GHz APs from scans dataframe for train and test

syl_scans_train = syl_scans_train.drop(syl_5ghz_aps_names, axis=1)
syl_scans_test = syl_scans_test.drop(syl_5ghz_aps_names, axis=1)

In [81]:
# check if any rows contain only 100 first 23 columns

num_of_24_aps = len(building_ap_info["SYL"]["2.4"])
# loop over rows
for idx, row in syl_scans_train.iterrows():
    # check if only contains 100s
    if np.all(row[:num_of_24_aps] == 100):
        print(idx)
        # drop the row
        syl_scans_train.drop(idx, inplace=True)

for idx, row in syl_scans_test.iterrows():
    # check if only contains 100s
    if np.all(row[:num_of_24_aps] == 100):
        print(idx)
        # drop the row
        syl_scans_test.drop(idx, inplace=True)

2962
2963
2964
2965
2966
2967
2968


In [82]:
# reset indices 

syl_scans_train = syl_scans_train.reset_index(drop=True)
syl_scans_test = syl_scans_test.reset_index(drop=True)

In [83]:
len(syl_scans_train)

8873

In [84]:
# define function to split data into files         

def split_scans_into_files(scans_dataframe, num_of_24_aps):

    scans = []
    for idx, row in scans_dataframe.iterrows():
        # get the x and y coordinate
        x, y = row["ECoord"], row["NCoord"]
        # get floorID
        floorID = row["FloorID"]
        scans.append([x, y])
    
    # create dataframe from scans
    scans_coords = pd.DataFrame(scans, columns=["x", "y"])

    scans = scans_dataframe.iloc[:, :num_of_24_aps]
    
    # rename columns
    scans.columns = [f"AP_{i}" for i in range(num_of_24_aps)]
    

    return scans, scans_coords

scans_train, scans_coords_train = split_scans_into_files(syl_scans_train, num_of_24_aps)
scans_test, scans_coords_test = split_scans_into_files(syl_scans_test, num_of_24_aps)


scans_test.to_csv("../data/raw/SYLtest_scans.csv", index=False)
scans_coords_test.to_csv("../data/raw/SYLtest_scans_coords.csv", index=False)
scans_train.to_csv("../data/raw/SYLtrain_scans.csv", index=False) 
scans_coords_train.to_csv("../data/raw/SYLtrain_scans_coords.csv", index=False)



In [85]:
# load HCXY building
hcxy_scans_train = pd.read_csv("../SODIndoorLoc-main/HCXY/Training_HCXY_AP_30.csv")
hcxy_scans_test = pd.read_csv("../SODIndoorLoc-main/HCXY/Testing_HCXY_AP.csv")

# # get hcxy 5 ghz ap names

# hcxy_5ghz_ap_names = [ key for key in building_ap_info["HCXY"]["5"].keys()]

# # drop the 5 GHZ APs from the scans
# hcxy_scans_train = hcxy_scans_train.drop(hcxy_5ghz_ap_names, axis=1)
# hcxy_scans_test = hcxy_scans_test.drop(hcxy_5ghz_ap_names, axis=1)

# check if any rows contain only 100s

num_of_24_aps = len(building_ap_info["HCXY"]["2.4"])

for idx, row in hcxy_scans_train.iterrows():
    if np.all(row[:num_of_24_aps] == 100):
        print(idx)
        # drop row
        hcxy_scans_train.drop(idx, inplace=True)

# repeat for test
for idx, row in hcxy_scans_test.iterrows():
    if np.all(row[:num_of_24_aps] == 100):
        print(idx)
        # drop row
        hcxy_scans_test.drop(idx, inplace=True)

# reset index
hcxy_scans_train.reset_index(drop=True, inplace=True)
hcxy_scans_test.reset_index(drop=True, inplace=True)

scans_train, scans_coords_train = split_scans_into_files(hcxy_scans_train, num_of_24_aps)
scans_test, scans_coords_test = split_scans_into_files(hcxy_scans_test, num_of_24_aps)

# save the scans
scans_train.to_csv("../data/raw/HCXYtrain_scans.csv", index=False)
scans_test.to_csv("../data/raw/HCXYtest_scans.csv", index=False)

# save the coordinates
scans_coords_train.to_csv("../data/raw/HCXYtrain_scan_coords.csv", index=False)
scans_coords_test.to_csv("../data/raw/HCXYtest_scan_coords.csv", index=False)


In [86]:
# repeat for CETC331

cetc331_scans_train = pd.read_csv("../SODIndoorLoc-main/CETC331/Training_CETC331.csv")
cetc331_scans_test = pd.read_csv("../SODIndoorLoc-main/CETC331/Testing_CETC331.csv")

# get cetc3311 5 ghz ap names
cetc331_5ghz_ap_names = [key for key in building_ap_info["CETC331"]["5"]]

# drop 5 ghz aps from scans
cetc331_scans_train = cetc331_scans_train.drop(cetc331_5ghz_ap_names, axis=1)
cetc331_scans_test = cetc331_scans_test.drop(cetc331_5ghz_ap_names, axis=1)

# check if any rows contain only 100s

num_of_24_aps = len(building_ap_info["CETC331"]["2.4"])

for idx, row in cetc331_scans_train.iterrows():
    if np.all(row[:num_of_24_aps] == 100):
        print(idx)
        # drop row
        cetc331_scans_train.drop(idx, inplace=True)

# repeat for test
for idx, row in cetc331_scans_test.iterrows():
    if np.all(row[:num_of_24_aps] == 100):
        print(idx)
        # drop row
        cetc331_scans_test.drop(idx, inplace=True)
        

# reset indices
cetc331_scans_train.reset_index(drop=True, inplace=True)
cetc331_scans_test.reset_index(drop=True, inplace=True)

scans_train, scans_coords_train = split_scans_into_files(cetc331_scans_train, num_of_24_aps)
scans_test, scans_coords_test = split_scans_into_files(cetc331_scans_test, num_of_24_aps)

# save the scans
scans_train.to_csv("../data/raw/CETC331train_scans.csv", index=False)
scans_test.to_csv("../data/raw/CETC331test_scans.csv", index=False)

# save the coordinates
scans_coords_train.to_csv("../data/raw/CETC331train_scan_coords.csv", index=False)
scans_coords_test.to_csv("../data/raw/CETC331test_scan_coords.csv", index=False)

In [87]:
# drop FloorID, BuildingID, SceneID, UserID, PhoneID, Sample

In [88]:
syl_scans_train

,MAC125,MAC112,MAC47,MAC22,MAC226,MAC79,MAC18,MAC208,MAC12,MAC81,...,MAC32,MAC8,ECoord,NCoord,FloorID,BuildingID,SceneID,UserID,PhoneID,SampleTimes
0,100,100,100,-55,100,100,-74,100,100,100,...,100,-49,19.03,20.05,4,3,1,1,1,1
1,100,100,100,-55,100,100,-74,100,100,100,...,100,-49,19.03,20.05,4,3,1,1,1,2
2,100,100,100,-55,100,100,-74,100,100,100,...,100,-49,19.03,20.05,4,3,1,1,1,3
3,100,100,100,-55,100,100,-74,100,100,100,...,100,-49,19.03,20.05,4,3,1,1,1,4
4,100,100,100,-55,100,100,-74,100,100,100,...,100,-49,19.03,20.05,4,3,1,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8868,-76,100,100,100,-67,100,100,-51,100,100,...,100,100,82.43,3.25,4,3,3,2,2,26
8869,-76,100,100,100,-67,100,100,-51,100,100,...,100,100,82.43,3.25,4,3,3,2,2,27
8870,-76,100,100,100,-67,100,100,-51,100,100,...,100,100,82.43,3.25,4,3,3,2,2,28
8871,-76,100,100,100,-67,100,100,-51,100,100,...,100,100,82.43,3.25,4,3,3,2,2,29
